In [71]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna

In [39]:
test_data = pd.read_csv('/Users/baconbaker/Documents/Studium/ANM/anm-project/data/test_data/esb.csv')
train_data = pd.read_csv('/Users/baconbaker/Documents/Studium/ANM/anm-project/data/train_data/esb.csv')

print(test_data.head())
print(train_data.head())

  service_name     start_time  avg_time  num  succee_num  succee_rate
0      osb_001  1606881600000    0.6214  361         361       1.0000
1      osb_001  1606881660000    0.5861  343         343       1.0000
2      osb_001  1606881720000    5.0125  126         122       0.9683
3      osb_001  1606881780000    9.3487   70          70       1.0000
4      osb_001  1606881840000    1.4756  375         375       1.0000
  serviceName      startTime  avg_time  num  succee_num  succee_rate
0     osb_001  1588521600000    0.5691  347         347          1.0
1     osb_001  1588521660000    0.5813  354         354          1.0
2     osb_001  1588521720000    0.5397  363         363          1.0
3     osb_001  1588521780000    0.6190  387         387          1.0
4     osb_001  1588521840000    0.4909  387         387          1.0


In [40]:
#Reformatting train data
train_data['start_time'] = train_data['startTime']

In [41]:
def get_onehot(df):
    df['start_time'] = pd.to_datetime(df['start_time'], unit='ms', origin='unix')
    df['start_hour'] = df["start_time"].dt.strftime("%H")
    df['start_minute'] = df["start_time"].dt.strftime("%M")
    return df

def split(df):
    df_x = df[['start_hour', 'start_minute']].astype('int32')
    df_y = df[['num', 'avg_time']]
    return df_x, df_y

In [42]:
train_data = get_onehot(train_data)
train_x, train_y = split(train_data)

test_data = get_onehot(test_data)
test_x, test_y = split(test_data)

In [75]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('depth', 2, 20),
        'eta': trial.suggest_float('eta', .8, 2),
        'objective': 'reg:squaredlogerror'}

    param['eval_metric'] = 'rmsle'
    evallist = [(dtest, 'eval'), (dtrain, 'train')]

    num_round = trial.suggest_int('rounds', 10, 300)
    model = xgb.train(param, dtrain, num_round, evallist, evals_result=progress)
    return progress['eval']['rmsle'][-1]
    

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=5000)

param = study.best_params  # E.g. {'x': 2.002108042}
print(param)

[I 2020-12-03 18:22:15,813] A new study created in memory with name: no-name-009f3c48-9640-45c4-8493-4a9f69d7e4be


[0]	eval-rmsle:0.28167	train-rmsle:0.02782
[1]	eval-rmsle:0.28579	train-rmsle:0.02234
[2]	eval-rmsle:0.28518	train-rmsle:0.01948
[3]	eval-rmsle:0.28540	train-rmsle:0.01826
[4]	eval-rmsle:0.28523	train-rmsle:0.01737
[5]	eval-rmsle:0.28527	train-rmsle:0.01683
[6]	eval-rmsle:0.28561	train-rmsle:0.01593
[7]	eval-rmsle:0.28552	train-rmsle:0.01531
[8]	eval-rmsle:0.28558	train-rmsle:0.01474
[9]	eval-rmsle:0.28552	train-rmsle:0.01426
[10]	eval-rmsle:0.28533	train-rmsle:0.01387
[11]	eval-rmsle:0.28522	train-rmsle:0.01344
[12]	eval-rmsle:0.28537	train-rmsle:0.01302
[13]	eval-rmsle:0.28536	train-rmsle:0.01284
[14]	eval-rmsle:0.28530	train-rmsle:0.01262
[15]	eval-rmsle:0.28546	train-rmsle:0.01229
[16]	eval-rmsle:0.28552	train-rmsle:0.01198
[17]	eval-rmsle:0.28541	train-rmsle:0.01173
[18]	eval-rmsle:0.28535	train-rmsle:0.01157
[19]	eval-rmsle:0.28533	train-rmsle:0.01148
[20]	eval-rmsle:0.28534	train-rmsle:0.01130
[21]	eval-rmsle:0.28536	train-rmsle:0.01111
[22]	eval-rmsle:0.28548	train-rmsle:0.0109

[I 2020-12-03 18:22:16,170] Trial 0 finished with value: 0.285775 and parameters: {'depth': 8, 'eta': 1.2957166662218937, 'rounds': 175}. Best is trial 0 with value: 0.285775.


[0]	eval-rmsle:0.27466	train-rmsle:0.05198
[1]	eval-rmsle:0.30297	train-rmsle:0.05398
[2]	eval-rmsle:0.27786	train-rmsle:0.04255
[3]	eval-rmsle:0.29719	train-rmsle:0.04180
[4]	eval-rmsle:0.28060	train-rmsle:0.03546
[5]	eval-rmsle:0.29262	train-rmsle:0.03371
[6]	eval-rmsle:0.28320	train-rmsle:0.02717
[7]	eval-rmsle:0.28784	train-rmsle:0.02357
[8]	eval-rmsle:0.28466	train-rmsle:0.01910
[9]	eval-rmsle:0.28570	train-rmsle:0.01630
[10]	eval-rmsle:0.28502	train-rmsle:0.01383
[11]	eval-rmsle:0.28578	train-rmsle:0.01204
[12]	eval-rmsle:0.28597	train-rmsle:0.01093
[13]	eval-rmsle:0.28584	train-rmsle:0.00959
[14]	eval-rmsle:0.28566	train-rmsle:0.00872
[15]	eval-rmsle:0.28573	train-rmsle:0.00796
[16]	eval-rmsle:0.28579	train-rmsle:0.00758
[17]	eval-rmsle:0.28585	train-rmsle:0.00694
[18]	eval-rmsle:0.28586	train-rmsle:0.00635
[19]	eval-rmsle:0.28596	train-rmsle:0.00604
[20]	eval-rmsle:0.28589	train-rmsle:0.00577
[21]	eval-rmsle:0.28592	train-rmsle:0.00536
[22]	eval-rmsle:0.28596	train-rmsle:0.0050

[185]	eval-rmsle:0.28592	train-rmsle:0.00144
[186]	eval-rmsle:0.28592	train-rmsle:0.00144
[187]	eval-rmsle:0.28592	train-rmsle:0.00144
[188]	eval-rmsle:0.28592	train-rmsle:0.00144
[189]	eval-rmsle:0.28592	train-rmsle:0.00144
[190]	eval-rmsle:0.28592	train-rmsle:0.00144
[191]	eval-rmsle:0.28592	train-rmsle:0.00144


...

[I 2020-12-03 18:23:48,591] Trial 1199 finished with value: 0.284494 and parameters: {'depth': 3, 'eta': 1.336637620862231, 'rounds': 23}. Best is trial 827 with value: 0.283819.


[0]	eval-rmsle:0.28203	train-rmsle:0.02732
[1]	eval-rmsle:0.28586	train-rmsle:0.02526
[2]	eval-rmsle:0.28471	train-rmsle:0.02436
[3]	eval-rmsle:0.28440	train-rmsle:0.02395
[4]	eval-rmsle:0.28476	train-rmsle:0.02355
[5]	eval-rmsle:0.28427	train-rmsle:0.02334
[6]	eval-rmsle:0.28434	train-rmsle:0.02316
[7]	eval-rmsle:0.28436	train-rmsle:0.02295
[8]	eval-rmsle:0.28424	train-rmsle:0.02278
[9]	eval-rmsle:0.28441	train-rmsle:0.02260
[10]	eval-rmsle:0.28442	train-rmsle:0.02246
[11]	eval-rmsle:0.28434	train-rmsle:0.02233
[12]	eval-rmsle:0.28443	train-rmsle:0.02227
[13]	eval-rmsle:0.28420	train-rmsle:0.02209
[14]	eval-rmsle:0.28419	train-rmsle:0.02199


[I 2020-12-03 18:23:48,645] Trial 1200 finished with value: 0.284189 and parameters: {'depth': 3, 'eta': 1.2729691044768399, 'rounds': 15}. Best is trial 827 with value: 0.283819.


[0]	eval-rmsle:0.28022	train-rmsle:0.03056
[1]	eval-rmsle:0.28705	train-rmsle:0.02588
[2]	eval-rmsle:0.28476	train-rmsle:0.02423
[3]	eval-rmsle:0.28549	train-rmsle:0.02337
[4]	eval-rmsle:0.28478	train-rmsle:0.02285
[5]	eval-rmsle:0.28452	train-rmsle:0.02254
[6]	eval-rmsle:0.28471	train-rmsle:0.02205
[7]	eval-rmsle:0.28473	train-rmsle:0.02177
[8]	eval-rmsle:0.28451	train-rmsle:0.02150
[9]	eval-rmsle:0.28494	train-rmsle:0.02122
[10]	eval-rmsle:0.28444	train-rmsle:0.02099
[11]	eval-rmsle:0.28481	train-rmsle:0.02075
[12]	eval-rmsle:0.28480	train-rmsle:0.02056
[13]	eval-rmsle:0.28457	train-rmsle:0.02029
[14]	eval-rmsle:0.28457	train-rmsle:0.02009
[15]	eval-rmsle:0.28492	train-rmsle:0.01991
[16]	eval-rmsle:0.28484	train-rmsle:0.01975
[17]	eval-rmsle:0.28490	train-rmsle:0.01958
[18]	eval-rmsle:0.28494	train-rmsle:0.01945
[19]	eval-rmsle:0.28498	train-rmsle:0.01933
[20]	eval-rmsle:0.28485	train-rmsle:0.01919
[21]	eval-rmsle:0.28502	train-rmsle:0.01908
[22]	eval-rmsle:0.28500	train-rmsle:0.0189

[I 2020-12-03 18:23:48,739] Trial 1201 finished with value: 0.285663 and parameters: {'depth': 4, 'eta': 1.3942077409722853, 'rounds': 36}. Best is trial 827 with value: 0.283819.


In [74]:
dtrain = xgb.DMatrix(train_x, label=train_y[['avg_time']])
dtest = xgb.DMatrix(test_x, label=test_y[['avg_time']])

param = {
    'max_depth': 3,
    'eta': 1.6487,
    'objective': 'reg:squaredlogerror'}

param['eval_metric'] = 'rmsle'
evallist = [(dtest, 'eval'), (dtrain, 'train')]
progress = {}

num_round = 60
model = xgb.train(param, dtrain, num_round, evallist, evals_result=progress)
print(progress['eval']['rmsle'][-1])

[0]	eval-rmsle:0.27700	train-rmsle:0.04058
[1]	eval-rmsle:0.29411	train-rmsle:0.03439
[2]	eval-rmsle:0.28225	train-rmsle:0.02807
[3]	eval-rmsle:0.28831	train-rmsle:0.02596
[4]	eval-rmsle:0.28372	train-rmsle:0.02454
[5]	eval-rmsle:0.28657	train-rmsle:0.02395
[6]	eval-rmsle:0.28475	train-rmsle:0.02347
[7]	eval-rmsle:0.28601	train-rmsle:0.02327
[8]	eval-rmsle:0.28449	train-rmsle:0.02304
[9]	eval-rmsle:0.28526	train-rmsle:0.02290
[10]	eval-rmsle:0.28475	train-rmsle:0.02271
[11]	eval-rmsle:0.28490	train-rmsle:0.02261
[12]	eval-rmsle:0.28423	train-rmsle:0.02247
[13]	eval-rmsle:0.28443	train-rmsle:0.02236
[14]	eval-rmsle:0.28444	train-rmsle:0.02214
[15]	eval-rmsle:0.28472	train-rmsle:0.02206
[16]	eval-rmsle:0.28490	train-rmsle:0.02194
[17]	eval-rmsle:0.28505	train-rmsle:0.02177
[18]	eval-rmsle:0.28501	train-rmsle:0.02166
[19]	eval-rmsle:0.28509	train-rmsle:0.02157
[20]	eval-rmsle:0.28502	train-rmsle:0.02150
[21]	eval-rmsle:0.28497	train-rmsle:0.02141
[22]	eval-rmsle:0.28492	train-rmsle:0.0213